In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print(f'Успешное подключение к базе по пути {path}')
    except Error as e:
        print(f'Ошибка {e}')
    return connection

In [3]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        #print('Запрос успешно выполнен')
    except Error as e:
        print(f'Ошибка {e}')

In [4]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [1]:
base_name = 'DB/tester_with_searchwords.sqlite'
connection = create_connection(base_name)

NameError: name 'create_connection' is not defined

In [6]:
create_users_table = '''
CREATE TABLE IF NOT EXISTS data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    annotation TEXT NOT NULL,
    registration_number TEXT NOT NULL       
);
'''
execute_query(connection, create_users_table)

In [7]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS keywords(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  work_registration_number TEXT NOT NULL, 
  keyword TEXT NOT NULL, 
  FOREIGN KEY (work_registration_number) REFERENCES data (registration_number)
);
"""
execute_query(connection, create_posts_table)

In [8]:
if base_name == 'DB/tester_with_searchwords.sqlite':
    create_key_search_table = '''
    CREATE TABLE IF NOT EXISTS keysearch(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        word TEXT NOT NULL,
        work_registration_number TEXT NOT NULL,
        FOREIGN KEY (work_registration_number) REFERENCES data (registration_number)
    )
    '''
    create_keys_table = '''
    CREATE TABLE IF NOT EXISTS allwords(
        if INTEGER PRIMARY KEY AUTOINCREMENT,
        ДОДЕЛАТЬ
    )
    '''
    execute_query(connection, create_key_search_table)

In [9]:
import pandas as pd

cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table_name in tables:
    if table_name[0] != 'sqlite_sequence':
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), connection)
        print(table_name)
        for col in table.columns:
            print('\t' + col)
        print()

data
	id
	name
	annotation
	registration_number

keywords
	id
	work_registration_number
	keyword

keysearch
	id
	word
	work_registration_number



In [10]:
import pymorphy2
from tqdm import tqdm
import string
from flashtext import KeywordProcessor
import re


def get_all_scrap_simbols():
    with open('Data/trash_simbols.txt', 'r', encoding='UTF-8') as f:
        trash_simbols = f.read()
    return trash_simbols

morph = pymorphy2.MorphAnalyzer()


with open('Data/predlogi.txt', 'r', encoding='utf-8') as f:
    predlogi = f.readline()
predlogi = predlogi.split(',')
#predlogi.extend(get_all_scrap_simbols())

#for ch in string.punctuation:
#    predlogi.append(ch)

punctuation_processor = KeywordProcessor()
for word in predlogi:
    punctuation_processor.add_keyword(word, ' ')


with open('Data/nioktr_2021_all_months.json', 'r', encoding="utf-8") as f:
    file_len = len(f.readlines())
file1 = open('Data/nioktr_2021_all_months.json', 'r', encoding="utf-8")


def get_rid_of_brackets(text):
    bracket_pos_left = text.find('"')
    while bracket_pos_left != -1:
        bracket_pos_right = text.find('"', bracket_pos_left)
        if bracket_pos_right == -1:
            break
        else:
            text = text.replace('"', '«', 1)
            text = text.replace('"', '»', 1)
    return text


def replase_punctuation(input):
    replased = re.sub(get_all_scrap_simbols(), ' ', input)
    return punctuation_processor.replace_keywords(replased)

'''
def has_numbers(input):
    return any(char.isdigit() for char in input)
'''

def make_base():
    works = []
    keyword_flag = False
    registration_flag = False
    count = 0
    #while True:
    #for i in tqdm(range(500)):
    #for i in tqdm(range(2364859)):            
    for i in tqdm(range(file_len)):  
        line = file1.readline()
        if not line:
            break
        if "registration_number" in line:
            registration_flag = True
            if 'АААА' in line:
                length = 33
            else:
                length = 24                
            left = line.find('"registration_number"') + length
            right = line.rfind('"')            
            registration_number = line[left:right]
        if "name" in line and registration_flag == True:
            left = line.find('"name": "') + 9
            right = line.rfind('"')
            s = get_rid_of_brackets(line[left:right])
            new_s = ''   

            #corrected = punctuation_processor.replace_keywords(s)
            corrected = replase_punctuation(s)

            for word in corrected.split():
                if word.find(',') >= 0 or word.find(';') >= 0 or word.find('.') >= 0:
                    word = word[:-1]
                new_s = new_s + ' ' + morph.parse(word)[0].normal_form
            works.append([new_s[1:], [], '', registration_number])
            count += 1            
        if ']' in line and keyword_flag == True:
            works[count-1][1].extend(temp)
            keyword_flag = False   
            registration_flag = False
        if keyword_flag == True and registration_flag == True:
            left = line.find('"')
            right = line.rfind('"')
            s = get_rid_of_brackets(line[left+1:right]) 
            s = s.split(',')
            for block in s:
                #corrected = punctuation_processor.replace_keywords(block) 
                corrected = replase_punctuation(block)
                new_s = ''
                for word in corrected.split():
                    new_s = new_s + ' ' + morph.parse(word)[0].normal_form
                temp.append(new_s[1:])
            #s = ' '.join(s)  
        if "keyword_list" in line:
            temp = []
            keyword_flag = True
        if "annotation" in line and registration_flag == True:            
            left = line.find('"annotation": "') + 15
            right = line.rfind('"')
            s = get_rid_of_brackets(line[left:right])                          
            #corrected = punctuation_processor.replace_keywords(s)   
            corrected = replase_punctuation(s)
            new_s = ''
            for word in corrected.split():
                if word.find(',') >= 0 or word.find(';') >= 0 or word.find('.') >= 0:
                    word = word[:-1]
                new_s = new_s + ' ' + morph.parse(word)[0].normal_form
            works[-1][2] = new_s[1:]
        #print(count)    
    all_words = {}
    for i, block in enumerate(tqdm(works)):
        all_text = set(f'{block[0]} {block[2]} {" ".join(block[1])}'.split())            
        for word in all_text:
            if len(word) > 2:
                #if has_numbers(word):
                value = all_words.get(word)
                if value != None:
                    value.append(block[3])
                else:
                    value = [block[3]]
                all_words.update({word : value})

        
                               
    return works, all_words

result, all_words = make_base()

100%|██████████| 16622/16622 [00:02<00:00, 7999.39it/s]


In [11]:
print(len(all_words))
print(len(result))

79571
16622


In [12]:
insert_data = f"""
INSERT INTO data (name, annotation, registration_number) VALUES 
"""    
insert_keywords = f"""
INSERT INTO keywords (work_registration_number, keyword)
VALUES
"""

for i, block in enumerate(tqdm(result)):
    insert_data += f'("{block[0]}", "{block[2]}", "{block[3]}"),'
    for keyword in block[1]:
        insert_keywords += f'({block[3]}, "{keyword}"),'
    if i % 8000 == 0:
        insert_data = insert_data[:-1]
        insert_keywords = insert_keywords[:-1]
        execute_query(connection, insert_data)
        execute_query(connection, insert_keywords)
        
        insert_data = f"""
        INSERT INTO data (name, annotation, registration_number) VALUES 
        """    
        insert_keywords = f"""
        INSERT INTO keywords (work_registration_number, keyword)
        VALUES
        """

if len(insert_data) > 82:
    insert_data = insert_data[:-1]
    insert_keywords = insert_keywords[:-1]
    execute_query(connection, insert_data)
    execute_query(connection, insert_keywords)


insert_serachkeys = f'''
INSERT INTO keysearch(word, work_registration_number)
VALUES 
'''
for i, (word, value) in enumerate(tqdm(all_words.items())):
    for reg_number in value:
        insert_serachkeys += f'("{word}", "{reg_number}"),'        

insert_serachkeys = insert_serachkeys[:-1]
execute_query(connection, insert_serachkeys)  

100%|██████████| 79571/79571 [07:29<00:00, 177.17it/s] 


In [13]:
import time

In [23]:
tim = time.time()

temp = '''
SELECT id, name from data
WHERE id < 10
'''
real_request = '''
SELECT data.id, data.name, keywords.keyword
FROM keywords INNER JOIN data on data.id = keywords.work_id
WHERE keywords.keyword = 'дериватография'
'''

custom_request = '''
SELECT data.id, data.name, keywords.keyword
FROM keywords INNER JOIN data on data.id = keywords.work_id
WHERE keywords.keyword = 'нанооптика'
'''

count_request = '''
SELECT COUNT (DISTINCT data.name) AS "Number of keywords"
FROM keywords INNER JOIN data on data.id = keywords.work_id
WHERE keywords.keyword = "нанооптика"
'''

count_like_request = '''
SELECT data.name
FROM keywords LEFT OUTER JOIN data on data.registration_number = keywords.work_registration_number
WHERE instr(keywords.keyword, 'нейтронный') OR instr(data.name, 'нейтронный') OR instr(data.annotation, 'нейтронный')
LIMIT 5
'''

count_keys_request_2 = '''
SELECT COUNT (DISTINCT word) FROM keysearch AS "Number of keywords"
'''

count_keys_request_1 = '''
SELECT COUNT (word) FROM keysearch AS "Number of keywords"
'''

def get_all_works_by_word(word):
    word = morph.parse(word)[0].normal_form
    key_search_request = f'''
    SELECT data.id, data.name
    FROM keysearch INNER JOIN data ON data.registration_number = keysearch.work_registration_number
    WHERE keysearch.word = '{word}'
    ORDER BY data.id
    LIMIT 5
    '''
    return key_search_request
'''
#WHERE keywords.keyword = 'нанооптика'
#WHERE instr(keywords.keyword, 'нанооптик')

'''

get_all_keys_request = f'''
SELECT data.name, keysearch.word
FROM keysearch INNER JOIN data ON data.registration_number = keysearch.work_registration_number
WHERE keysearch.word = 'дериватография'
LIMIT 5
'''

key = 'нейтронный'
cursor = connection.cursor()

#posts = cursor.execute(count_keys_request_2).fetchall()
#posts = cursor.execute(count_keys_request_1).fetchall()
#posts = cursor.execute(get_all_works_by_word(key)).fetchall()
posts = cursor.execute(count_like_request).fetchall()

print(f'Заданное слово - {key}')
for post in posts:
    print(post)

print(time.time()-tim)

Заданное слово - нейтронный
(None,)
(None,)
(None,)
(None,)
(None,)
0.4649984836578369
